<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-1/blob/main/ex11_G%C3%A9rer_des_valeurs_NULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ex11 - Gérer des valeurs NULL

Les exemples de données dans les tables de demo.db3 présentés précédemment sont tous exacts et complets. 

Chaque ligne a une valeur pour chaque attribut. 

Cependant, les données réelles ne sont généralement pas aussi propres et ordonnées. Vous trouverez souvent des valeurs NULL dans certaines tables.

Les valeurs nulles dans une base de données peuvent causer quelques maux de tête. De plus, les descriptions dans les normes SQL sur la façon de gérer les NULL semblent ambiguës. 

Il n'apparaît pas clairement dans les documents de normes comment les NULL doivent être traités dans toutes les [circonstances] (https://www.sqlite.org/nulls.html).

Parfois, nous pouvons en fait éviter les NULL en définissant la contrainte NOT NULL lorsque nous créons une table. 

Cependant, il convient de garder à l'esprit que rendre les champs NOT NULL ne fonctionne pas toujours et pourrait créer plus de maux de tête qu'il ne guérit. Toutes les valeurs nulles ne signifient pas qu'il y a un problème avec les données.

SQLite NULL est le terme utilisé pour représenter une valeur manquante. 

Une valeur NULL dans une table est une valeur dans un champ qui semble vide. Cependant, une valeur NULL ne doit pas simplement être considérée comme 0 (zéro) ou une chaîne vide comme «». Il s'agit d'une valeur de vide ou non définie.

Ce notebook présentera:
- Comment faite un **DROP** sur une table **IF EXISTS**
- Comment faire un **CREATE** d'une nouvelle table à partir d'une table existante
- Comment faire un ***UPDATE*** d'une table avec une condition WHERE
- Comment faire un ***COUNT NULL*** avec ***IS NULL***
- Comment donner des valeurs par défaut à ***NULL*** avec la fonction SQLite *** COALESCE ***

In [1]:
%load_ext sql

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

Il a été précisé que demo.db3 est un extrait d'une modélisation hydrologique. Par conséquent, les données de chaque table sont ordonnées et complètes sans valeurs NULL. Cependant, nous pouvons créer une table avec des valeurs NULL pour la démonstration.

In [3]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne vous souvenez pas des tables présentes dans la database de démonstration, vous pouvez toujours utiliser la commande suivante pour les retrouver.

In [4]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Création d'une table avec des valeurs NULL à partir d'une table existante

Prenons l'exemple du tableau des watershed_yearly (ie bassins versants_année).
- ***Tout d'abord, créons une table de sauvegarde***

> L'instruction SQLite CREATE TABLE AS est utilisée pour créer une table à partir d'une table existante en copiant les colonnes de la table existante.

In [5]:
%%sql sqlite://
DROP TABLE  IF EXISTS watershed_yearly_bk;
CREATE TABLE watershed_yearly_bk AS SELECT * FROM watershed_yearly

Done.
Done.


[]

Vérifions rapidement la table de backup

In [6]:
%%sql sqlite://
SELECT YR, PREC_mm 
FROM watershed_yearly_bk
---LIMIT 3

Done.


YR,PREC_mm
1981,895.6051025390625
1982,884.670654296875
1983,816.6605224609375
1984,867.5743408203125
1985,637.7255249023438
1986,733.8412475585938
1987,1007.8944702148438
1988,895.8466186523438
1989,930.10546875
1990,751.4553833007812


- ***Ensuite, définissons certaines des valeurs à NULL***

> La requête SQLite UPDATE est utilisée pour modifier les enregistrements existants dans une table. 

> Nous utilisons ici la clause WHERE avec la requête UPDATE pour mettre à jour seulement les lignes sélectionnées, car bien sûr, sans clause where, toutes les lignes seraient mises à jour.

In [7]:
%%sql sqlite://
UPDATE watershed_yearly_bk
SET PREC_mm = NULL
WHERE
    PREC_mm < 850.0

18 rows affected.


[]

### 3. Recherche des NULL

Les valeurs nulles ne peuvent pas être déterminées avec un =. 

Nous devons utiliser les instructions IS NULL ou IS NOT NULL pour identifier les valeurs nulles. 

Donc, pour obtenir tous les enregistrements sans valeurs de précipitations enregistrées, nous pourrions exécuter la requête :

In [8]:
%%sql sqlite://
SELECT  YR, PREC_mm
FROM watershed_yearly_bk
WHERE PREC_mm IS NULL

Done.


YR,PREC_mm
1983,None
1985,None
1986,None
1990,None
1994,None
1995,None
1996,None
1997,None
1998,None
1999,None


Comptage des années "NULLs"

In [9]:
%%sql sqlite://
SELECT  COUNT(YR) AS MISSING
FROM watershed_yearly_bk
WHERE PREC_mm IS NULL

Done.


MISSING
18


C'est évidemment le nombre d'enregistrements que nous avions modifiés

### 4. Comment gérer les NULL

Les valeurs NULL peuvent être ambiguës et gênantes car elles sont identifiées différemment selon les sources de données. 

Il peut y avoir des valeurs NULL pour un certain nombre de raisons telles que les observations qui n'ont pas été enregistrées ou du fait de la corruption des données.


En général, il existe deux stratégies principales pour gérer les valeurs NULL :



#### 4.1 Ne pas utiliser de lignes avec des valeurs NULL

Cette stratégie est assez simple car nous pouvons toujours filtrer les données avec une condition ***WHERE IS NOT NULL***. 

Cependant, dans la pratique, les données ne seraient pas représentatives ou même utilisées du tout, si le nombre de NULL est trop important.


In [10]:
%%sql sqlite://
SELECT  YR, PREC_mm
FROM watershed_yearly_bk
WHERE PREC_mm IS NOT NULL

Done.


YR,PREC_mm
1981,895.6051025390625
1982,884.670654296875
1984,867.5743408203125
1987,1007.8944702148438
1988,895.8466186523438
1989,930.10546875
1991,984.4703369140625
1992,907.9463500976562
1993,1057.7733154296875
2005,855.0092163085938


*Calculate the counts of NULLs, NOT_NULLs and total*. Keep in mind that the ***COUNT*** function will neglet NULL values.

* Calcul du nombre de **NULL**, de **NOT NULL** et du **TOTAL** 

Gardez à l'esprit que la fonction **COUNT** ne comptabilisera pas les valeurs **NULL**.

In [11]:
%%sql sqlite://
SELECT SUM(CASE WHEN PREC_mm IS NULL THEN 1 else 0 END) COUNT_NULLs,
       COUNT(PREC_mm) COUNT_NOT_NULLs,
       COUNT(YR) AS TOTAL
From watershed_yearly_bk

Done.


COUNT_NULLs,COUNT_NOT_NULLs,TOTAL
18,12,30


#### 4.2 Remplacer les valeurs NULL par des valeurs précises ***

Il est recommandé de vérifier d'abord le document décrivant la base de données afin de s'assurer que les colonnes NULL (colonnes autorisées à avoir des valeurs NULL) sont bien documentées et que l'on a bien précisé ce que signifie une valeur NULL d'un point de vue métier avant de remplacer les valeurs NULL par des valeurs précises.

Le SQLite offre une manière plus élégante de gérer les valeurs NULL. 

Cela consiste à utiliser la fonction COALESCE () qui accepte deux ou plusieurs arguments, et renvoie le premier argument non nul dans une valeur par défaut spécifiée si elle est nulle. 

Si tous les arguments sont NULL, la fonction COALESCE renvoie NULL.

Ce qui suit illustre la syntaxe de la fonction COALESCE: <br>
*** COALESCE (paramètre1, paramètre2,…) ***; <br>

Ici, nous voulons que tous les NULL de PREC_mm soient traités comme la moyenne climatologique de NOT NULL.


***Calculate the mean nof NON-NULLs***

In [12]:
%%sql sqlite://
SELECT avg(PREC_mm)
From watershed_yearly_bk

Done.


avg(PREC_mm)
936.1221313476562


***Remplacement des NULL par la moyenne ci-dessus des NON-NULL***

In [13]:
%%sql sqlite://
SELECT YR, COALESCE(PREC_mm, 936.122131348) as Precipitation
From watershed_yearly_bk

Done.


YR,Precipitation
1981,895.6051025390625
1982,884.670654296875
1983,936.122131348
1984,867.5743408203125
1985,936.122131348
1986,936.122131348
1987,1007.8944702148438
1988,895.8466186523438
1989,930.10546875
1990,936.122131348


### Conclusion 

Le traitement des valeurs NULL est une tâche complexe. 

Il est préférable de demander l'assistance d'experts du domaine Métier, sauf si vous savez déjà très clairement à quoi servent les valeurs NULL.
